Create a dataset from 3 numpy files

Import numpy file for each shape

In [ ]:
import numpy as np
import os
# circle = np.load("../data/circle/circle_data_preprocess.npy")    # (N1, 128, 128)  # (N1, 128, 128)
# square = np.load("../data/square/square_data_preprocess.npy")    # (N2, 128, 128)
# triangle = np.load("../data/mini_dataset/triangle/triangle_data_preprocess.npy")# (N3, 128, 128)
base_dir = "D:/data_saxs_LPS/reduced_dataset"

path_circle = os.path.join(base_dir,"circle/first_channel.npy")
path_square = os.path.join(base_dir,"square/first_channel.npy")
path_triangle = os.path.join(base_dir,"triangle/first_channel.npy")



# path_circle = "D:/data_saxs_LPS/reduced_dataset/circle/circle_data_preprocess.npy"
# path_square = "D:/data_saxs_LPS/reduced_dataset/square/square_data_preprocess.npy"
# path_triangle = "D:/data_saxs_LPS/reduced_dataset/triangle/triangle_data_preprocess.npy"

circle = np.load(path_circle)
square = np.load(path_square)    # (N2, 128, 128)
triangle = np.load(path_triangle)# (N3, 128, 128

# circle = np.load("D:/data_saxs_LPS/reduced_dataset/circle/first_channel.npy")
# square = np.load("D:/data_saxs_LPS/reduced_dataset/square/first_channel.npy")    # (N2, 128, 128)
# triangle = np.load("D:/data_saxs_LPS/reduced_dataset/triangle/first_channel.npy")# (N3, 128, 128



print(circle.shape)
print(square.shape)
print(triangle.shape)

Create corresponding labels.
Create 3 numpy array:
- 1 filled with 0 for circles, the shape is `len(circle)`
- 1 filled with 1 for squares, the shape is `len(square)`
- 1 filled with 2 for triangles, the shape is `len(triangle)`


In [ ]:
y_circle = np.zeros(len(circle), dtype=int)           # classe 0
y_square = np.ones(len(square), dtype=int)            # classe 1
y_triangle = np.full(len(triangle), 2, dtype=int) 

print(y_circle.shape)
print(y_square.shape)
print(y_triangle.shape)

Concatenante the 3 arrays to pass from shapes (x,) (y,) (z,) to (x+y+z,)

In [ ]:
y = np.concatenate([y_circle, y_square, y_triangle], axis=0)
print(y.shape)

Concatenate the 3 numpy array to pass from dimensions (x, 128, 128) (y, 128, 128) (y, 128, 128) to 1 array of shape (x+y+z,128, 128)

In [ ]:
X_before_resize = np.concatenate([circle, square, triangle], axis=0) 
print(X_before_resize.shape)

Reduce images (Let the raw image, juste resize)

In [ ]:
from skimage.transform import resize
import numpy as np

def resize_with_padding(img, target_size=128):
    h, w = img.shape
    scale = target_size / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    img_resized = resize(img, (new_h, new_w),
                         order=1,
                         preserve_range=True,
                         anti_aliasing=True)
    # padding pour obtenir un carré
    pad_h = (target_size - new_h) // 2
    pad_w = (target_size - new_w) // 2
    padded = np.zeros((target_size, target_size), dtype=img_resized.dtype)
    padded[pad_h:pad_h+new_h, pad_w:pad_w+new_w] = img_resized
    return padded

# Pour tout ton dataset (N,H,W)
print(X_before_resize.shape)
X = np.array([resize_with_padding(img) for img in X_before_resize])
print(X.shape)  # (N,128,128)


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.imshow(X[0,:, :],cmap='gray')

Keras need data in the shape 

In [ ]:
print(X.shape)
X = X[..., np.newaxis]
print(X.shape)

OPTIONAL: Set random seed to (for reproducibility)

In [ ]:
import random
import tensorflow as tf
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

Shuffle with scikit learn to avoid learning on blocks

In [ ]:
from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=seed)
print(y)

Split dataset in train and test

In [ ]:
from sklearn.model_selection import train_test_split

# Suppose que X et y sont déjà construits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,         # 20% en test
    random_state=seed,       # pour reproductibilité
    stratify=y             # pour garder la proportion des classes
)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


Create a CNN 2D Model

In [ ]:
import numpy as np
from tensorflow.keras import layers, models

# --- ton modèle ---
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    #layers.Conv2D(32, (3,3), activation='relu', input_shape=(1083,1035,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
# --- entraînement ---
history = model.fit(
    X_train, y_train,          # tes données d'entraînement
    epochs=20,                 # nombre d'epochs
    batch_size=64,             # taille de batch
    validation_split=0.2,      # fraction du train utilisée pour validation
    verbose=1                  # affiche les métriques
)

# --- tracé des courbes de métriques ---
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Predictions 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# --- prédiction sur le jeu de test ---
y_pred_proba = model.predict(X_test)              # probabilités (nb_images, 3)
y_pred = np.argmax(y_pred_proba, axis=1)          # classes prédites (0,1,2)

# --- accuracy ---
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy sur le jeu de test : {acc:.3f}")

# --- matrice de confusion ---
cm = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :")
print(cm)

# --- affichage joli ---
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['circle','square','triangle'],
            yticklabels=['circle','square','triangle'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title(f'Matrice de confusion (Accuracy {acc:.2%})')
plt.tight_layout()
plt.show()


In [ ]:

# model : ton modèle Keras
def model_size_and_params(model, dtype_size=4):
    params = model.count_params()
    # approx size in bytes (float32 = 4 bytes)
    size_bytes = params * dtype_size
    size_mb = size_bytes / (1024**2)
    return params, size_mb

In [ ]:
import time
def file_size_mb(path):
    """Retourne la taille d’un fichier en MB"""
    return os.path.getsize(path) / (1024 ** 2)

circle_size   = file_size_mb(path_circle)
square_size   = file_size_mb(path_square)
triangle_size = file_size_mb(path_triangle)


def measure_inference_time(model, input_shape, n_warmup=10, n_run=100):
    # generate random input (batch size 1)
    x = np.random.rand(1, *input_shape).astype(np.float32)
    # warmup
    for _ in range(n_warmup):
        _ = model.predict(x, verbose=0)
    # timed runs
    t0 = time.time()
    for _ in range(n_run):
        _ = model.predict(x, verbose=0)
    t1 = time.time()
    avg_ms = (t1 - t0) / n_run * 1000
    return avg_ms

print(f"Taille circle:   {circle_size:.2f} MB")
print(f"Taille square:   {square_size:.2f} MB")
print(f"Taille triangle: {triangle_size:.2f} MB")
print(f"Taille totale:   {circle_size + square_size + triangle_size:.2f} MB")

In [ ]:
# assume model, X_train, folder_path exist
params, model_mb = model_size_and_params(model)
print("Params:", params)
print("Model size (MB):", model_mb)
print(f"Taille totale:   {circle_size + square_size + triangle_size:.2f} MB")
avg_infer_ms = measure_inference_time(model, X_train.shape[1:])
print("Inference ms:", avg_infer_ms)
# training time demo (1 epoch)

t0 = time.time()
model.fit(X_train[:128], y_train[:128], epochs=1, batch_size=8, verbose=1)
print("1 epoch time (s):", time.time()-t0)
import psutil, os
print("Process RSS (MB):", psutil.Process(os.getpid()).memory_info().rss / 1024**2)


For pre-processed data:   
<pre>
Params: 3705475
Model size (MB): 14.135265350341797
Taille totale:   230.25 MB
Inference ms: 710.1784157752991
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.8729 - loss: 0.3561
1 epoch time (s): 3.1816084384918213
Process RSS (MB): 1528.75390625
</pre>
---
<pre>
Params: 3705475
Model size (MB): 14.135265350341797
Taille totale:   230.25 MB
Inference ms: 561.4975476264954
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 182ms/step - accuracy: 0.3554 - loss: 1.4452
1 epoch time (s): 8.540453910827637
Process RSS (MB): 1283.8359375
</pre>
---
---
For raw data

<pre>
Params: 283187843
Model size (MB): 1080.2758903503418
Taille totale:   7876.23 MB
Inference ms: 940.4475092887878
16/16 ━━━━━━━━━━━━━━━━━━━━ 219s 11s/step - accuracy: 0.4024 - loss: 18972995584.0000
1 epoch time (s): 232.40875720977783
Process RSS (MB): 11297.3125
</pre>